In [ ]:
# default_exp uhi

# Uhi

> API details.

In [ ]:
#export
from ufro_uhi_plot.map import Map
import pandas as pd
import numpy as np
import os

In [ ]:
#export
class Uhi:
    """Represents an image(s) of urban heat island(s)

    Attributes:
      path_file: string
      station_label: string
      datetime_label: string
      temperature_label: string
      latitude_label: string
      longitude_label: string
    """
    def __init__(self,path_file,station_label,datetime_label,temperature_label,latitude_label,longitude_label):
      self.path_file = path_file
      self.station_label = station_label
      self.datetime_label = datetime_label
      self.temperature_label = temperature_label
      self.latitude_label = latitude_label
      self.longitude_label = longitude_label
      self.df=pd.read_csv(self.path_file,decimal=",",parse_dates=True,index_col=datetime_label,usecols=[station_label,datetime_label,temperature_label,latitude_label,longitude_label])
      self.dfToFloat()
  
    def dfToFloat(self):
      self.df=self.df.dropna(subset=[self.temperature_label])
      self.df[self.longitude_label] = self.df[self.longitude_label].astype(np.float)
      self.df[self.latitude_label] = self.df[self.latitude_label].astype(np.float)
      self.df[self.temperature_label] = self.df[self.temperature_label].astype(np.float)
  
    def plotImagesPerDay(self,dfDay,outputFolder,extent,hasBorders=None,prefixFile="",hasLimits=False,transects=[],Z_UNIT="°C",otherDf_id=None,otherDf_name=None,mapbox_username="",mapbox_token=""):
      hours=dfDay.groupby(dfDay.index)
      dfListDay=[]
      for hour in hours.groups.keys():
        df_hour=hours.get_group(str(hour))
        if len(df_hour)>4:
          print(hour)
          dfListDay.append(df_hour)
          otherDf = None
          if otherDf_name is not None and otherDf_id is not None:
            otherDf_df = df_hour[df_hour[self.station_label]==otherDf_id]
            otherDf = OtherDf(otherDf_df,otherDf_id,otherDf_name)
            df_hour = df_hour[df_hour[self.station_label]!=otherDf_id]
          varMap = Map(df_hour,transects=transects,otherDf=otherDf,mapbox_username=mapbox_username,mapbox_token=mapbox_token)
          vMin=None
          vMax=None
          if hasLimits:
            vMin=dfDay[self.temperature_label].min()
            vMax=dfDay[self.temperature_label].max()
          varMap.plot(extent,hasBorders=hasBorders,file=outputFolder+"/{} {}".format(prefixFile,hour),vMin=vMin,vMax=vMax,contour="contourLine",
                      ID_LABEL=self.station_label,LON_LABEL=self.longitude_label,
                      LAT_LABEL=self.latitude_label,Z_LABEL=self.temperature_label,Z_UNIT=Z_UNIT)
          
      return dfListDay
  
    def plotImagesPerDaysArray(self,outputFolder,daysArray,extent,hasBorders=None,prefixFile="",hasLimits=False,transects=[],Z_UNIT="°C",otherDf_id=None,otherDf_name=None,mapbox_username="",mapbox_token=""):
      """

      Input:
      outputFolder: string of folder where the images of urban heat islands are saved.
      daysArray: array of strings of days to plot the urban heat islands.
      extent: array of 4 coordinates of the extent of the map.
      hasBorders: geojson to border the urban heat island in a map. (default:None)
      prefixFile: string of the output prefix filename. (default:'')
      hasLimits: (default:False)
      transects: (default:[])
      Z_UNIT: unit of measure to be displayed in colobar. (default: '°C')
      otherDf_id: id of the temperature station that will not be included in the UHI's triangulation but will be displayed. (default: None)
      otherDf_name: name of the temperature station that will not be included in the UHI's triangulation but will be displayed. (default: None)
      mapbox_username: mapbox's username to plot background map. (default: '')
      mapbox_token: mapbox's token to plot background map. (default: '')

      Output: None
      """
      if not os.path.exists(outputFolder):
        try:
            os.makedirs(outputFolder)
        except OSError:
            print ("Creation of the directory %s failed" % outputFolder)
        else:
            print ("Successfully created the directory %s" % outputFolder)

      for day in daysArray:
        print("Plotting UHI Map of ",day)
        df1=self.df.loc[np.in1d(self.df.index.date, pd.to_datetime([day]).date)]
        print("...")
        self.plotImagesPerDay(df1,outputFolder,extent,hasBorders=hasBorders,prefixFile=prefixFile,hasLimits=hasLimits,transects=[],Z_UNIT=Z_UNIT,otherDf_id=otherDf_id,otherDf_name=otherDf_name,mapbox_username=mapbox_username,mapbox_token=mapbox_token)
        print("Plot completed",day)
class OtherDf:
  def __init__(self,df,id,name):
    self.df = df
    self.id = id
    self.name = name

In [ ]:
show_doc(Uhi.plotImagesPerDaysArray)